<a href="https://colab.research.google.com/github/Twinkle-gawri/CNN/blob/main/Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# low level api in pytorch

In [1]:
import torch
import torch.nn as nn #nn library of torch
from torch.utils.data import DataLoader
import torchvision

In [2]:
train_set = torchvision.datasets.MNIST('/data',
                           train=True,
                           download=True,
                           transform=torchvision.transforms.ToTensor())
test_set = torchvision.datasets.MNIST('/data',
                           train=False,
                           download=True,
                           transform=torchvision.transforms.ToTensor())

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 9.91M/9.91M [00:00<00:00, 15.8MB/s]


Extracting /data/MNIST/raw/train-images-idx3-ubyte.gz to /data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 28.9k/28.9k [00:00<00:00, 485kB/s]


Extracting /data/MNIST/raw/train-labels-idx1-ubyte.gz to /data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 1.65M/1.65M [00:00<00:00, 4.40MB/s]


Extracting /data/MNIST/raw/t10k-images-idx3-ubyte.gz to /data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 4.54k/4.54k [00:00<00:00, 5.49MB/s]

Extracting /data/MNIST/raw/t10k-labels-idx1-ubyte.gz to /data/MNIST/raw



In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
device

device(type='cuda')

In [7]:
model = nn.Sequential(
    nn.Flatten(),
    nn.Linear(28*28,512) ,  #wx+b , not define number of neurons but we define neurons of this and next layers
    nn.ReLU(),
    nn.Linear(512,256),  # previous layer,next layer
    nn.ReLU(),
    nn.Linear(256,10)
)
# keras -- defined in list
# here linear layer is defined here

In [8]:
# 2nd way to define
layers = [nn.Flatten(),
    nn.Linear(28*28,512) ,
    nn.ReLU(),
    nn.Linear(512,256),
    nn.ReLU(),
    nn.Linear(256,10)]
model1 = nn.Sequential(*layers)

In [10]:
model = torch.compile(model).to(device)  # better way to initialize weight and bias , not important over here

In [12]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=0.001)  # optim is the lib , important thing -- model.parameter -- optimizer kinko optimize karega

In [18]:
def batch_accuracy(output,y_true,N): # no inbuild class, we have to make
  # prob of last layers -- output toh hum argmax
  y_pred = torch.argmax(output,dim=1)  # output have prob,dtype,shape -- we want only 1st col
  correct = (y_pred == y_true).sum().item() #.item to return scaler
  return correct/N

In [19]:
train_loader = DataLoader(train_set,batch_size=64,shuffle=True,num_workers=4) # num_worker -- how many will it work parallely
test_loader = DataLoader(test_set,batch_size=64,shuffle=False,num_workers=4)

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [20]:
# we have to do custom training over here only
def training():
  loss = 0
  accuracy = 0
  model.train() # to initialize the shape of gradient, not imp
  for image,label in train_loader: #image size, label
    image = image.to(device)
    label = label.to(device)
    output = model(image)  # model ki last layer ki actn fun not need to define
    batch_loss = loss_fn(output,label)
    optimizer.zero_grad()
    batch_loss.backward()
    optimizer.step()  # adjust the parameter accordingly
    loss += batch_loss.item()
    accuracy += batch_accuracy(output,label,label.shape[0])
  print(f'Accuracy is {accuracy/len(train_loader)}') # no. of batches , train_loader.datasets will tell number of examples
  print(f'Loss is {loss/len(train_loader)}')

In [21]:
def validation():
  loss = 0
  accuracy = 0
  model.eval()
  with torch.no_grad(): # matlab kisi bhi cheez ka gradient nhi compute karna
    for image,label in test_loader:
      image = image.to(device)
      label = label.to(device)
      output = model(image)
      batch_loss = loss_fn(output,label)
      loss += batch_loss.item()
      accuracy += batch_accuracy(output,label,label.shape[0])
  print(f'Accuracy is {accuracy/len(test_loader)}')
  print(f'Loss is {loss/len(test_loader)}')

In [22]:
epochs = 10
for i in range(epochs):
  print(f'Epoch {i+1}')
  training()
  validation()

Epoch 1
Accuracy is 0.9324027185501066
Loss is 0.2300318783409258


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Accuracy is 0.9665605095541401
Loss is 0.10506867192860597
Epoch 2


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Accuracy is 0.972664578891258
Loss is 0.08897554981924378
Accuracy is 0.9745222929936306
Loss is 0.08046432691837578
Epoch 3
Accuracy is 0.9822594616204691
Loss is 0.05636914278532484
Accuracy is 0.9770103503184714
Loss is 0.06973177712940118
Epoch 4
Accuracy is 0.9868236940298507
Loss is 0.04001654923592172
Accuracy is 0.9767117834394905
Loss is 0.08207671126733744
Epoch 5
Accuracy is 0.9894389658848614
Loss is 0.03262429725281667
Accuracy is 0.9791998407643312
Loss is 0.06893535708221844
Epoch 6
Accuracy is 0.9917044243070362
Loss is 0.025610432139771464
Accuracy is 0.9811902866242038
Loss is 0.07075031232775154
Epoch 7
Accuracy is 0.9934201759061834
Loss is 0.020718702321149113
Accuracy is 0.9809912420382165
Loss is 0.07313924722567283
Epoch 8
Accuracy is 0.9938532782515992
Loss is 0.018005090183337968
Accuracy is 0.9796974522292994
Loss is 0.07855511538787865
Epoch 9
Accuracy is 0.9948027718550106
Loss is 0.01566676310802243
Accuracy is 0.9837778662420382
Loss is 0.0714214108367740